In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import config

In [2]:
wiki = requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
wiki_soup = BeautifulSoup(wiki.content)


# <a rel="nofollow" class="external text" href="https://www.nyse.com/quote/XNYS:MMM">MMM</a>
symbols = (
    [x.get_text() 
           for x in 
           wiki_soup.find_all(attrs={'class':'external text'}) if x.get_text()!='reports'][:505]
)

In [3]:
def get_companies(symbols):
    companies = []
    for symbol in symbols:
        try:
            request = requests.get(f'https://datafied.api.edgar-online.com/v2/companies?primarysymbols={symbol}&appkey={config.edgar_apiKey}')
            response = request.json()
            companies.append({x['field']:x['value'] for x in response['result']['rows'][0]['values']})
        except:
            continue
    return companies

def corefinancials(primary_symbol):
    request = requests.get(f"""https://datafied.api.edgar-online.com/v2/corefinancials/ann?primarysymbols={primary_symbol}&numperiods=1&appkey={config.edgar_apiKey}""")
    return request.json()

def get_coreFinancials(symbols):
    dfs = []
    for symbol in symbols:
        try:
            contents = corefinancials(symbol)
            financials = {x['field']:x['value'] for x in contents['result']['rows'][0]['values']}
            temp_df = pd.DataFrame(financials,index=[financials['cik']])
            dfs.append(temp_df)
        except:
            continue
    return pd.concat(dfs,sort=True)

In [ ]:
#df=get_coreFinancials(symbols)

In [4]:
df = get_coreFinancials(symbols[:50])

In [5]:
df = df.append(get_coreFinancials(symbols[50:100]))

In [6]:
df = df.append(get_coreFinancials(symbols[100:200]))

In [7]:
df = df.append(get_coreFinancials(symbols[200:300]))

In [8]:
df = df.append(get_coreFinancials(symbols[300:400]))

In [9]:
df = df.append(get_coreFinancials(symbols[400:505]))

In [97]:
df

,amended,audited,capitalexpenditures,cashandcashequivalents,cashcashequivalentsandshortterminvestments,cashfromfinancingactivities,cashfrominvestingactivities,cashfromoperatingactivities,cfdepreciationamortization,changeinaccountsreceivable,...,totalcurrentassets,totalcurrentliabilities,totalliabilities,totallongtermdebt,totalreceivablesnet,totalrevenue,totalshorttermdebt,totalstockholdersequity,treasurystock,usdconversionrate
0000066740,False,true,-1.577000e+09,2.853000e+09,3.233000e+09,-6.701000e+09,2.220000e+08,6.439000e+09,1.488000e+09,-305000000.0,...,1.370900e+10,7.244000e+09,2.670400e+10,1.341100e+10,5.020000e+09,3.276500e+10,1.211000e+09,9.796000e+09,-2.962600e+10,1.0
0000001800,False,true,-1.394000e+09,3.844000e+09,4.086000e+09,-1.039100e+10,-1.356000e+09,6.300000e+09,3.310000e+09,-190000000.0,...,1.463200e+10,9.012000e+09,3.664900e+10,1.935900e+10,5.182000e+09,3.057800e+10,2.070000e+08,3.052400e+10,-9.962000e+09,1.0
0001551152,False,true,-6.380000e+08,7.289000e+09,8.061000e+09,-1.439600e+10,-1.006000e+09,1.342700e+10,1.765000e+09,-591000000.0,...,1.694500e+10,1.723900e+10,6.779800e+10,3.500200e+10,5.384000e+09,3.275300e+10,5.308000e+09,-8.446000e+09,-2.410800e+10,1.0
0000815094,False,true,-4.400400e+07,1.210210e+08,4.916980e+08,-5.577500e+07,-1.164550e+08,2.521970e+08,1.412100e+07,-22023000.0,...,6.771970e+08,1.059980e+08,1.078810e+08,NaN,9.080900e+07,7.694320e+08,NaN,9.368900e+08,-1.388520e+08,1.0
0001467373,False,true,-5.990090e+08,6.126853e+09,6.130166e+09,-3.767171e+09,-1.755576e+09,6.626953e+09,8.927600e+08,NaN,...,1.545060e+10,1.106190e+10,1.538087e+10,1.624700e+07,8.095071e+09,4.321501e+10,6.411000e+06,1.440901e+10,-1.388376e+09,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0001524472,False,true,-2.370000e+08,2.960000e+08,2.960000e+08,-4.000000e+07,-6.430000e+08,5.860000e+08,2.610000e+08,NaN,...,2.094000e+09,1.389000e+09,4.454000e+09,2.051000e+09,1.031000e+09,5.207000e+09,2.570000e+08,2.768000e+09,-4.870000e+08,1.0
0001041061,False,true,-2.340000e+08,2.920000e+08,2.920000e+08,-2.620000e+09,3.130000e+08,1.176000e+09,1.370000e+08,-66000000.0,...,1.207000e+09,1.301000e+09,1.205600e+10,9.751000e+09,5.610000e+08,5.688000e+09,3.210000e+08,-7.926000e+09,NaN,1.0
0001136869,False,true,-4.390000e+08,5.428000e+08,5.428000e+08,-1.302200e+09,-4.166000e+08,1.747400e+09,1.040500e+09,NaN,...,4.427400e+09,2.421300e+09,1.285550e+10,8.413700e+09,1.275800e+09,7.932900e+09,5.250000e+08,1.127130e+10,-6.721700e+09,1.0
0000109380,False,true,-1.290000e+08,1.233000e+09,2.694000e+09,1.596000e+09,-2.706000e+09,1.176000e+09,1.930000e+08,NaN,...,NaN,NaN,6.116800e+10,7.240000e+08,4.631200e+10,2.821000e+09,NaN,7.578000e+09,NaN,1.0


In [96]:
df.to_csv('/Users/schlinkertc/Flatiron/projects/classification/files/new_data.csv',index_label='cik')